# Munging together .csv data #

It's very common to see tabular or field-based data (pretty much anything you could imagine putting in a spreadsheet) distributed in the form of .csv or .tsv (comma- or tab-separated value) files. While you can open these delimited files with a spreadsheet program like Microsoft's Excel, they are simply plain-text files, which means they can be opened by any text editor, making them lightweight and platform-independent: perfect for data interchange.

In this exercise, we're going to start with two different .csv files that have different pieces of information we want about some of the same books. The first file is the .csv with ESTC bibliographic data about books printed by William Bowyer (which you exported from MarcEdit). The second file provides metadata for TEI-encoded texts created as part of the ECCO-TCP (Text Creationship Partnership) project.

Each file has something that the other doesn't. The ESTC file lets us know which eighteenth-century books were printed by Bowyer but doesn't give us any information about how to get the text. The TCP metadata file tells us how to get the text, but doesn't tell us anything about which texts were printed by Bowyer. But both files include some common data--crucially, an ESTC citation number--that we can use to connect the two sets of information.

We'll read the file of Bowyer-connected ESTC records to get their ESTC numbers, look for those ESTC numbers in the file of TCP metadata, and produce a new file that adds a TCP identifer to our ESTC metadata.

*Note:* This code was written to prioritize breaking things down into small, relatively discrete steps. There are ways it could be made more concise.

## First, where are you running this notebook? ##

These notebooks were first written for use in L-100 (Digital Approaches to Bibliography and Book history) at Rare Book School in 2018. If you were a student in that course, you received an image of a virtual machine running Ubuntu Linux, along with the files that the various Jupyter Notebooks work with. If you've come to this notebook from a link at GitHub to MyBinder, you won't have those files, but I've added them to GitHub. 

NOTE: If you're running this notebook from the RBSDigitalApproaches2018 virtual machine, you'll need to be sure that you're running `jupyter notebook` with either the `marc` or `web-scraping` virtualenv activated. If you run into errors below related to missing libraries, quite the notebook and Jupyter Notebooks, go back to your terminalk and execute `workon marc` or `workon web-scraping` to activate that virtualenv. (If another virtualenv is active, you may have to execute `deactivate` first.)

The first thing we need to do is to determine what file paths to use in the rest of the code below. Simply comment or uncomment the appropriate line by adding or deleting the octothorpe (#) at the beginning of the line, then run the cell.

In [ ]:
# If you're running this notebook from the RBSDigitalApproaches2018 virtual machine
# in VirtualBox, make sure the line below is uncommented (that's the default for this
# notebook.)
location = 'rbs_virtual_machine'

# If you're running this notebook at MyBinder using a link from the GitHub repository,
# comment out line 4, above, and uncomment line 8, below:
#location = 'mybinder'

if location == 'rbs_virtual_machine' :
    input_path = '/media/sf_RBSDigitalApproaches/data/0611_Tuesday_data/'
    output_path = '/media/sf_RBSDigitalApproaches/output/'
elif location == 'mybinder' :
    input_path = 'data/'
    output_path = 'output/'
    # If you're running this notebook at MyBinder.org, the code on this page will end up
    # writing files to the "output" folder. To download the files you create, go to the 
    # Jupyter Notebook "Home" tab for the repository, enter the "output" folder, click 
    # on the filename of the file you created, then click the "Download" button.

## Step 1: Get the ESTC numbers of texts printed by William Bowyer ##

We'll get the ESTC numbers of texts printed by William Bowyer from the .csv file we exported from MarcEdit earlier and save them in a list.

In [ ]:
# Import the csv library
import csv
# Create an empty list to hold our ESTC numbers
estc_nums = []

# A workaround to make this notebook available at MyBinder.org. If you're working in the
# RBSDigitalApproaches2018 virtual machine, we saved the Bowyer .csv file to "output".
# If you're note sure where your .csv file ended up, you can change your location in the
# code block above, then re-run it.
if location == 'rbs_virtual_machine' :
    path = output_path
elif location == 'mybinder' :
    path = input_path

# Open the .csv file of ESTC bibliographic data.
with open(path + 'Bowyer_from_ESTC-full.csv', 'r') as estcfile:
    # Initiate a csv DictReader
    estcreader = csv.DictReader(estcfile, delimiter=',', quotechar='"')
    # Read through the .csv file a line at a time
    for row in estcreader :
        # Get the ESTC number from the '001 cell
        estc_num = row['001']
        # Check to see if the ESTC number is already in our list of ESTC numbers...
        if estc_num not in estc_nums :
            # If it's not, add it to the list
            estc_nums.append(estc_num)

# Sort the list in place. No good reason. I'm just funny that way.
estc_nums.sort()

# Let's print our list and see what we have.
print(estc_nums)


## Step 2: Get TCP ids for the ESTC numbers we just found ##

We'll search through our second .csv file of ECCO-TCP metadata looking for the ESTC numbers we just found. When we find one of those ESTC numbers, we'll store it and its corresponding TCP identifier in a dictionary, with the ESTC number as the key and the TCP number as the value.

In [ ]:
# Create an empty dictionary to store our matched ESTC numbers and the TCP ids that correspond to them
estc_tcp = {}
# Open our .csv file of ECCO-TCP metadata
with open(input_path + 'ecco_tcp_ids.csv', 'r') as tcpfile :
    # Initiate a csv DictReader
    tcpreader = csv.DictReader(tcpfile, delimiter=',', quotechar='"')
    # Read through the .csv file a line at a time
    for row in tcpreader :
        # Find the ESTC number
        estc_num = row['ESTC_Number']
        # Check to see if this ESTC number is in the list of Bowyer ESTC numbers we created in step 1...
        if estc_num in estc_nums :
            # If it is, create an entry in our estc_tcp dictionary, with the ESTC number as the key and the 
            # TCP id as the value
            estc_tcp[estc_num] = row['TCP_Number']

# Let's see what we have.            
print(estc_tcp)

## Step 3: Write our matched ESTC data to a new .csv file, adding a column with the TCP id ##

Now it's time to write our matches to a file (we'll use this file later to identify TCP texts to download). Mostly, we're just going to copy relevant rows from one .csv file to another, but we need to add a new column. This led me to a solution that strikes me as actually a little confusing, which I'll try to explain in the comments. As always, there may well be a more "Pythonic" solution, but this is the one that occurred to me.

I've commented out several lines that perform the real business of this section so that we can see each of the steps involved:
1) Run the cell to see the fieldnames that get generated, then comment out line 17.
2) Uncomment line 34 and run the cell again to see results that we're constructing before we write out the file. Then comment out line 34 again.
3) Uncomment lines 18, 19, and 35 and run the cell again to actually write our new .csv file.

In [ ]:
# Open the .csv file of Bowyer records again (this is one of the inefficiencies of this script in its current
# form--we could probably figure out a way to do this all in one pass), and create a new .csv file to hold our
# output.
with open(path + 'Bowyer_from_ESTC-full.csv', 'r') as bibdata, \
open(output_path + 'Bowyer_TCP_texts.csv', 'w') as outfile :
    # Initiate a csv DictReader
    reader = csv.DictReader(bibdata, delimiter=',', quotechar='"')
    # Begin getting the fieldnames for our new file by getting the keys from the first row of the .csv file
    # (reader.next() gets the first row, .keys() gets the keys--DictReader reads each row and holds its 
    # values as a dictionary of key/value pairs). Dictionaries are, by nature, unsorted, so our keys would
    # come back in some random order. That wouldn't be a problem, but I've sorted the keys because otherwise
    # I'd be haunted by the untidiness of it all.
    fieldnames = sorted(reader.next().keys())
    # Add a column heading for our TCP ids by inserting a new item to our list of fieldnames at index [1] 
    # (i.e., make it the second item in the list)
    fieldnames.insert(1,'tcp_id')
    print(fieldnames)
    #writer = csv.DictWriter(outfile, delimiter=',', fieldnames = fieldnames)
    #writer.writeheader()
    
    # Read through the .csv file of Bowyer bibliographic data a line at a time
    for row in reader :
        # Get the ESTC number
        estc_num = row['001']
        # Check to see if that ESTC number is among the keys in our ecco_tcp dictionary...
        if estc_num in estc_tcp.keys() :
            # If it is, copy the row (that is, assign the entire row to a new variable called outrow). Keep in
            # mind that, because we're working with DictReader, our row is a dictionary rather than a list.
            outrow = row
            # Now, add a new key/value pair to our outrow dictionary, with 'tcp_id' as the key (to correspond to
            # the column heading we added in line 17), and the value in our estc_tcp dictionary that has this
            # this ESTC number as its key
            outrow['tcp_id'] = estc_tcp[estc_num]
            #print(outrow)
            #writer.writerow(outrow)    